# International Trade

## Imports

In [53]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import pmp_functions_v4 as pmp

import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

path = "../../Data_Ryan"

## Data

In [54]:
# --- Benchmark Data ---
benchmark_data = pd.read_excel(
    f"{path}/Equity Returns.xlsx",
    sheet_name = "WORLD - MXWO Index",
    index_col = 0,
    parse_dates = True
)

benchmark_data.index = pd.to_datetime(benchmark_data.index)

display(benchmark_data)

,Price (USD)
Date,
1970-01-30,94.2500
1970-02-27,96.9800
1970-03-31,97.0700
1970-04-30,87.8000
1970-05-29,82.0600
...,...
2025-07-31,8057.2246
2025-08-29,8269.7393
2025-09-30,8538.7900


In [55]:
# --- Load Factors Data ---
factors_data = pd.read_excel(
    f"{path}/Factors.xlsx",
    index_col = 0,
    parse_dates = True
)

factors_data.index = pd.to_datetime(factors_data.index, format='%Y%m')
factors_data.index = factors_data.index + pd.offsets.MonthEnd(0)
factors_data /= 100
display(factors_data)

# --- Riskfree Rate ---
riskfree = factors_data["RF"]
riskfree = riskfree.to_frame()
display(riskfree)

/var/folders/bl/ywj9s7j57_qf7pwls_sv_w9r0000gn/T/ipykernel_87170/2744343010.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  factors_data = pd.read_excel(


,Mkt-RF,SMB,HML,RF
1926-07-31,0.0289,-0.0255,-0.0239,0.0022
1926-08-31,0.0264,-0.0114,0.0381,0.0025
1926-09-30,0.0038,-0.0136,0.0005,0.0023
1926-10-31,-0.0327,-0.0014,0.0082,0.0032
1926-11-30,0.0254,-0.0011,-0.0061,0.0031
...,...,...,...,...
2025-06-30,0.0486,0.0083,-0.0160,0.0034
2025-07-31,0.0198,0.0027,-0.0127,0.0034
2025-08-31,0.0184,0.0387,0.0442,0.0038
2025-09-30,0.0339,-0.0184,-0.0105,0.0033


,RF
1926-07-31,0.0022
1926-08-31,0.0025
1926-09-30,0.0023
1926-10-31,0.0032
1926-11-30,0.0031
...,...
2025-06-30,0.0034
2025-07-31,0.0034
2025-08-31,0.0038
2025-09-30,0.0033


In [56]:
# --- Load Trade Data ---
trade_data = pd.read_excel(
    f"{path}/International Trade Data.xlsx",
    sheet_name = 'Normalized - Exports (%)',
    index_col = 0,
    parse_dates = True
)
trade_data.index = pd.to_datetime(trade_data.index)
trade_data.index = trade_data.index + pd.offsets.MonthEnd(0)

start_date = trade_data.index.min()
end_date = pd.to_datetime('2025-10-31')
monthly_index = pd.date_range(start = start_date, end = end_date, freq ='ME')

trade_weight = trade_data.reindex(monthly_index)

trade_weight = trade_weight.ffill()

display(trade_weight)

,UK,CH,JP,AU,EU,EM
1980-01-31,0.136825,0.023996,0.239271,0.047531,0.514998,0.037379
1980-02-29,0.136825,0.023996,0.239271,0.047531,0.514998,0.037379
1980-03-31,0.136825,0.023996,0.239271,0.047531,0.514998,0.037379
1980-04-30,0.136825,0.023996,0.239271,0.047531,0.514998,0.037379
1980-05-31,0.136825,0.023996,0.239271,0.047531,0.514998,0.037379
...,...,...,...,...,...,...
2025-06-30,0.100211,0.093618,0.108386,0.044831,0.454114,0.198840
2025-07-31,0.100211,0.093618,0.108386,0.044831,0.454114,0.198840
2025-08-31,0.100211,0.093618,0.108386,0.044831,0.454114,0.198840
2025-09-30,0.100211,0.093618,0.108386,0.044831,0.454114,0.198840


In [57]:
# --- Load Currency Prices ---
currency_data = pd.read_excel(
    f"{path}/FX Data.xlsx",
    sheet_name = 'SPOT',
    index_col = 0,
    parse_dates = True
)

currency_data = currency_data.rename(columns={
    'CHFUSD' : 'CH',
    'EURUSD' : 'EU',
    'JPYUSD' : 'US',
    'AUDUSD' : 'AU',
    'GBPUSD' : 'UK',
    'JPYUSD' : 'JP',
    'CNHUSD' : 'EM'
})

currency_data.index = pd.to_datetime(currency_data.index)
currency_data.index = currency_data.index + pd.offsets.MonthEnd(0)
currency_spot = currency_data

display(currency_spot)

,CH,EU,JP,AU,UK,EM
Date,,,,,,
1988-12-31,0.6658,NaN,0.007997,0.8555,1.8110,NaN
1989-01-31,0.6251,NaN,0.007663,0.8890,1.7520,NaN
1989-02-28,0.6431,NaN,0.007882,0.8015,1.7434,NaN
1989-03-31,0.6015,NaN,0.007532,0.8195,1.6855,NaN
1989-04-30,0.5976,NaN,0.007527,0.7942,1.6900,NaN
...,...,...,...,...,...,...
2025-06-30,1.2609,1.1787,0.006943,0.6581,1.3732,0.1397
2025-07-31,1.2311,1.1415,0.006634,0.6425,1.3207,0.1387
2025-08-31,1.2492,1.1686,0.006800,0.6540,1.3504,0.1404


## Global Variables

In [58]:
frequency = 1
t_cost = 0
# window = 12*20
short = True
beta_neutral = False
target_vol = 0.10
rf = riskfree
benchmark = benchmark_data

## Signal Generation

In [59]:
# --- Compute International Trade Signal ---
# 1. INVERT QUOTES where necessary (Make everything USD base)
# Example: If you have EURUSD, invert it to get USDEUR
# (Skip this if your data is already "Foreign per USD")

currency_spot_US_base = currency_spot.copy()
currency_spot_US_base['EU'] = 1 / currency_spot['EU'] 
currency_spot_US_base['CH'] = 1 / currency_spot['CH'] 
currency_spot_US_base['JP'] = 1 / currency_spot['JP'] 
currency_spot_US_base['AU'] = 1 / currency_spot['AU']
currency_spot_US_base['UK'] = 1 / currency_spot['UK'] 
currency_spot_US_base['EM'] = 1 / currency_spot['EM']
currency_spot_US_base['USD'] = 1.0

display('Base Currency: US (USD)')
display(currency_spot_US_base)

'Base Currency: US (USD)'

,CH,EU,JP,AU,UK,EM,USD
Date,,,,,,,
1988-12-31,1.501953,NaN,125.046893,1.168907,0.552181,NaN,1.0
1989-01-31,1.599744,NaN,130.497194,1.124859,0.570776,NaN,1.0
1989-02-28,1.554968,NaN,126.871352,1.247661,0.573592,NaN,1.0
1989-03-31,1.662510,NaN,132.766861,1.220256,0.593296,NaN,1.0
1989-04-30,1.673360,NaN,132.855055,1.259129,0.591716,NaN,1.0
...,...,...,...,...,...,...,...
2025-06-30,0.793084,0.848392,144.029958,1.519526,0.728226,7.158196,1.0
2025-07-31,0.812282,0.876040,150.738619,1.556420,0.757174,7.209805,1.0
2025-08-31,0.800512,0.855725,147.058824,1.529052,0.740521,7.122507,1.0


In [67]:
cross_rates_dict = {}

for new_base_currency in currency_spot_US_base.columns:
    # Formula: Divide the Whole DataFrame by the New Base Currency column
    # axis=0 aligns the division by index (dates)
    cross_rates_dict[new_base_currency] = currency_spot_US_base.div(currency_spot_US_base[new_base_currency], axis = 0).dropna()

display("Base Currency: EU (EUR)")
display(cross_rates_dict['EU'])

display("Base Currency: JP (JPY)")
display(cross_rates_dict['JP'])

'Base Currency: EU (EUR)'

,CH,EU,JP,AU,UK,EM,USD
Date,,,,,,,
2010-08-31,1.287179,1.0,106.751978,1.423759,0.826166,8.590786,1.2680
2010-09-30,1.339688,1.0,113.872881,1.409782,0.867524,9.029139,1.3634
2010-10-31,1.370039,1.0,112.132176,1.418099,0.869622,9.181698,1.3947
2010-11-30,1.302729,1.0,108.644351,1.354088,0.834276,8.638057,1.2983
2010-12-31,1.251660,1.0,108.609916,1.307925,0.857289,8.811060,1.3384
...,...,...,...,...,...,...,...
2025-06-30,0.934808,1.0,169.768112,1.791065,0.858360,8.437366,1.1787
2025-07-31,0.927220,1.0,172.068134,1.776654,0.864314,8.229993,1.1415
2025-08-31,0.935479,1.0,171.852941,1.786850,0.865373,8.323362,1.1686


'Base Currency: JP (JPY)'

,CH,EU,JP,AU,UK,EM,USD
Date,,,,,,,
2010-08-31,0.012058,0.009368,1.0,0.013337,0.007739,0.080474,0.011878
2010-09-30,0.011765,0.008782,1.0,0.012380,0.007618,0.079291,0.011973
2010-10-31,0.012218,0.008918,1.0,0.012647,0.007755,0.081883,0.012438
2010-11-30,0.011991,0.009204,1.0,0.012463,0.007679,0.079508,0.011950
2010-12-31,0.011524,0.009207,1.0,0.012042,0.007893,0.081126,0.012323
...,...,...,...,...,...,...,...
2025-06-30,0.005506,0.005890,1.0,0.010550,0.005056,0.049699,0.006943
2025-07-31,0.005389,0.005812,1.0,0.010325,0.005023,0.047830,0.006634
2025-08-31,0.005443,0.005819,1.0,0.010398,0.005036,0.048433,0.006800


In [68]:
# # 2. CALCULATE 1-YEAR RETURN (Pct Change)
currency_spot_inverted_chg = currency_spot_inverted.pct_change(12)

display('YoY% Inverted Spot')
display(currency_spot_inverted_chg)

# # 3. CALCULATE WEIGHTED BASKET CHANGE
# # This creates a single column representing the USD Index return
usd_basket_return = (
    (currency_spot_inverted_chg['EU'] * trade_weight['EU']) +
    (currency_spot_inverted_chg['CH'] * trade_weight['CH']) +
    (currency_spot_inverted_chg['JP'] * trade_weight['JP']) +
    (currency_spot_inverted_chg['AU'] * trade_weight['AU']) +
    (currency_spot_inverted_chg['UK'] * trade_weight['UK'])
)

display(usd_basket_return)

# # 4. CREATE THE SIGNAL (Depreciation)
# We want Positive values to mean Depreciation (Weak Dollar)
# because Weak Dollar is usually the "Buy" signal for this strategy.
fx_signal = -1 * usd_basket_return

display(fx_signal)

'YoY% Inverted Spot'

,CH,EU,JP,AU,UK,EM
Date,,,,,,
1988-12-31,NaN,NaN,NaN,NaN,NaN,NaN
1989-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1989-02-28,NaN,NaN,NaN,NaN,NaN,NaN
1989-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1989-04-30,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,-0.117535,-0.091117,-0.104710,0.013524,-0.079158,-0.019327
2025-07-31,-0.074811,-0.051599,0.004974,0.018210,-0.026577,-0.002163
2025-08-31,-0.057717,-0.054595,0.006029,0.034404,-0.027918,0.004274


1980-01-31         NaN
1980-02-29         NaN
1980-03-31         NaN
1980-04-30         NaN
1980-05-31         NaN
                ...   
2025-06-30   -0.071056
2025-07-31   -0.031743
2025-08-31   -0.030798
2025-09-30   -0.023898
2025-10-31   -0.032423
Length: 550, dtype: float64

1980-01-31         NaN
1980-02-29         NaN
1980-03-31         NaN
1980-04-30         NaN
1980-05-31         NaN
                ...   
2025-06-30    0.071056
2025-07-31    0.031743
2025-08-31    0.030798
2025-09-30    0.023898
2025-10-31    0.032423
Length: 550, dtype: float64